In [15]:
import json
import pandas as pd
from datetime import date
from cafu.metadata import path

In [16]:
path_datalake = path('datalake')

In [17]:
r = open(path_datalake+'/metadata.json', 'r')
metadata_datalake = json.load(r)

In [18]:
campeonatos = [
               [k1, k2] for k1 in list(metadata_datalake['jogos_ids'].keys()) 
                        for k2 in list(metadata_datalake['jogos_ids'][k1].keys())
                        if metadata_datalake['jogos_ids'][k1][k2] != 'failed'
              ]

In [ ]:
today = date.today()

In [49]:
prox_partidas = {c:{} for c in metadata_datalake['jogos_ids']}
for c in campeonatos:
    df = pd.read_csv(path_datalake+f'/jogos_ids/{c[0]}/{c[1]}.csv')
    if len(df[df['dates']>=str(today)]['dates'])==0:
        prox_partidas[c[0]][c[1]] = 'finalizado'
    else:
        prox_partidas[c[0]][c[1]] = min(df[df['dates']>=str(today)]['dates'])

In [50]:
prox_partidas

{'brasil': {'2021-2021': 'finalizado',
  '2020-2020': 'finalizado',
  '2019-2019': 'finalizado',
  '2018-2018': 'finalizado',
  '2017-2017': 'finalizado',
  '2016-2016': 'finalizado',
  '2015-2015': 'finalizado',
  '2014-2014': 'finalizado'},
 'espanha': {'2021-2022': '2022-03-11',
  '2020-2021': 'finalizado',
  '2019-2020': 'finalizado',
  '2018-2019': 'finalizado',
  '2017-2018': 'finalizado',
  '2016-2017': 'finalizado',
  '2015-2016': 'finalizado',
  '2014-2015': 'finalizado',
  '2013-2014': 'finalizado'}}

In [38]:
c = ['brasil', '2014-2014']
df = pd.read_csv(path_datalake+f'/jogos_ids/{c[0]}/{c[1]}.csv')
min(df[df['dates']>=str(today)]['dates'])

ValueError: min() arg is an empty sequence

In [1]:
from cafu import path

In [2]:
path('logs_cafu')

'/home/luan/futebol/logs_cafu/2022-03-10'

In [1]:
import pandas as pd
import json
import time
from cafu.utils.etl.partidas_campeonato import id_left_right
from cafu.utils.string import convert_str_var_time
from cafu.metadata.campeonatos_espn import campeonato_espn
from cafu.metadata.paths import path

In [2]:
campeonato, temporada = 'espanha', '2012-2013'

In [3]:
dados_campeonato = campeonato_espn(campeonato, temporada)
campeonato = dados_campeonato['nome']
id_inicial =dados_campeonato['id']
qt_jogos_rodada = dados_campeonato['qt_jogos_rodada']
qt_partidas_campeonato = 2*qt_jogos_rodada*(2*qt_jogos_rodada-1)
ids, partidas, dates = id_left_right(id_inicial, campeonato, qt_partidas_campeonato)

 14%|█▎        | 42/306 [03:13<20:14,  4.60s/it]


In [4]:
ids, partidas, dates = id_left_right(id_inicial+1, campeonato, qt_partidas_campeonato,
                                     left=False, partidas=partidas, dates=dates, ids=ids)

300it [22:53,  4.58s/it]                         


In [5]:
data = []
excedentes = len(partidas)%qt_jogos_rodada
for i in range(len(partidas)-excedentes):
    data.append([ids[i], dates[i], partidas[i]])
df = pd.DataFrame(data, columns=['jogo_id', 'dates', 'partida']).sort_values('jogo_id')
df.index = range(df.shape[0])

In [7]:
# criando coluna "rodada"
dates = []
for i in range(0,df.shape[0], qt_jogos_rodada):
    median_date = df.iloc[i:i+qt_jogos_rodada]['dates'].median()
    dates_0 = [median_date for i in range(qt_jogos_rodada)]
    dates = dates+dates_0

In [10]:
rodadas = [i for i in range(1,(qt_jogos_rodada*2-1)*2+1) for j in range(qt_jogos_rodada)]
len(rodadas)

306

In [11]:
df.shape

(342, 4)

In [8]:
df['median_dates'] = dates[:df.shape[0]]
df = df.sort_values(['median_dates'])
rodadas = [i for i in range(1,(qt_jogos_rodada*2-1)*2+1) for j in range(qt_jogos_rodada)]
df['rodada'] = rodadas[:df.shape[0]]
df = df.drop('median_dates',axis=1)
df = df.sort_values(['jogo_id'])

df[['time_casa','time_visitante']] = pd.DataFrame(df['partida'].values.tolist(), index= df.index)
df = df.drop('partida', axis=1)

ValueError: Length of values (306) does not match length of index (342)